In [1]:
from dotenv import load_dotenv, find_dotenv # type: ignore

load_dotenv(find_dotenv())

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

Vediamo che il modello non è in grado di risponderci poichè non è stato addestrato su tali dati. Per rispondere alla domanda deve avere accesso a dei tools che fanno la ricerca in tempo reale sul web

In [4]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm unable to provide real-time weather updates. For the most current weather information in Munich, please check a reliable weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 17, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-af102dff-0c81-4ea6-ad70-8cb31b1f8257-0', usage_metadata={'input_tokens': 17, 'output_tokens': 28, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Tool Defining

In [5]:
from langchain_core.tools import tool

# importiamo il decoratore e poi definiamo la funzione che il decoratore va a usare
# importante è fornire la descrizione per il tool, dato che questa viene usata dall'llm
# per identificare se llm ha bisongno del tool per rispondere alla domanda, e se 
# il tool è effettivamente adatto per rispondere alla domanda
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ['munich']:
        return "It's 15 degrees Celsius and cloudly."
    else: 
        return "It's 32 degrees Celsius and sunny."
    
# ora vedimao un esempio per la prenotazione di un ristorante all'esterno o
# all'interno e controllare se ci sono ancora posti a disposizione
@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unvailable."
    
# dobiamo fornire i tools all'LLM 
tools = [get_weather, check_seating_availability]

In [7]:
# utilizziamo bind_tools method per legare/associare i tools all'llm
# bind_tools è un metodo della calsse OpenAI che invia insieme alla query 
# i tool che saranno a disposizione per il modello
llm_with_tools = llm.bind_tools(tools)

In [8]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tFpWaCfk05blQVqJFt9dhW5A', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 82, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1243958d-c3fa-43f2-9aba-7d1267ef3333-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_tFpWaCfk05blQVqJFt9dhW5A', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 16, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning'

Non ci da nessuna risposta ma la cosa importante è che in tool_calls vediamo che il modello LLM vuole usare il tool **get_weather** con gli argomenti location uguale a Munich

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'call_tFpWaCfk05blQVqJFt9dhW5A',
  'type': 'tool_call'}]

In [22]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)

result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'call_0N5wt2DYgNPYjCaiKiQVSELk',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': 'call_0K8DZAoizckJRZQu4fgeZre2',
  'type': 'tool_call'}]

Ora vedimao come utilizzare tale tool_calls, che ci da/ che vuole fare l'LLM, in un workflow. Dovremo fornire una lista di messaggi che inviamo a OpenAI.

In [25]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available"
    )
]

llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [27]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'call_KaemY2rizhSSw4YQkE4KeThC',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': 'call_j09hlL9k0o5mC8001QhbsPBd',
  'type': 'tool_call'}]

In [28]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KaemY2rizhSSw4YQkE4KeThC', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_j09hlL9k0o5mC8001QhbsPBd', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 89, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9cb5434f-fb75-4

Vediamo che il content è ancora vuoto ma tool_calls vuole chiamare il tool get_weather.

Quindi il prossimo passo è quello di chiamare effetivamente il tool.

Possiamo fare ciò fornendo un **tool_mapping** così diamo un suggerimento di quali tools usare all'LLM.

In questo modo chiamiamo i tools noi stessi.

In [30]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability
}

In [31]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call['name'].lower()]
    tool_output = tool.invoke(tool_call['args']) 
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call['id']))

In [32]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KaemY2rizhSSw4YQkE4KeThC', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_j09hlL9k0o5mC8001QhbsPBd', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 89, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9cb5434f-fb75-4

Questi messagi, con le risposte delle chiamate ai tools, sono quelli che vogliamo inviare all'LLM in modo che costruisca la risposta finale all'utente.

In [33]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is 15 degrees Celsius and cloudy. Yes, we still have seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 176, 'total_tokens': 200, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-2f2a66c6-4a04-46cf-82a2-700d6c2ac5d1-0', usage_metadata={'input_tokens': 176, 'output_tokens': 24, 'total_tokens': 200, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})